In [15]:
import getpass
import os
import pandas as pd
from langchain_openai import OpenAIEmbeddings


In [16]:
# Load the environment variables
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [17]:
df = pd.read_csv("dataset/diseases_symptoms.tsv", sep="\t")

In [18]:
symptoms_each_disease = df["symptoms"].tolist()

In [19]:
import re

def clean_symptoms(symptoms_list):
  cleaned_symptoms = []
  for symptoms in symptoms_list:
      if isinstance(symptoms, list):
          symptoms = ','.join(symptoms)
      cleaned = re.sub(r',+', ',', symptoms).strip(', ').strip()
      if (len(cleaned) == 0):
        print("true")
      cleaned_symptoms.append(cleaned)

  return cleaned_symptoms

def seperate_symptoms(symptoms_each_disease):
  return [symptoms.split(', ') for symptoms in symptoms_each_disease]

cleaned_symptoms_each_disease = clean_symptoms(symptoms_each_disease)

seperated_symptoms_each_disease = seperate_symptoms(cleaned_symptoms_each_disease)


seperated_symptoms_each_disease[0]

['Abdomen',
 'Abdominal Pain',
 'Abnormal',
 'Acquired',
 'Acute',
 'Acute Coronary Syndrome',
 'Aerophagy',
 'Ageusia',
 'Agnosia',
 'Agraphia',
 'Akathisia',
 'Albuminuria',
 'Amblyopia',
 'Amnesia',
 'Angina',
 'Angina Pectoris',
 'Anisocoria',
 'Anomia',
 'Anorexia',
 'Anoxia',
 'Anterograde',
 'Anticipatory',
 'Aphasia',
 'Aphonia',
 'Apnea',
 'Apraxia',
 'Apraxias',
 'Arthralgia',
 'Articulation Disorders',
 'Asthenia',
 'Ataxia',
 'Athetosis',
 'Auditory Perceptual Disorders',
 'Babinski',
 'Back Pain',
 'Bilateral',
 'Birth Weight',
 'Blindness',
 'Body Weight',
 'Broca',
 'Brown-Sequard Syndrome',
 'Bulimia',
 'Cachexia',
 'Cardiac',
 'Cardiac Output',
 'Catatonia',
 'Central',
 'Cerebellar Ataxia',
 'Cerebrospinal Fluid Otorrhea',
 'Cerebrospinal Fluid Rhinorrhea',
 'Chest Pain',
 'Cheyne-Stokes Respiration',
 'Chills',
 'Chorea',
 'Colic',
 'Color Vision Defects',
 'Coma',
 'Communication Disorders',
 'Conduction',
 'Conductive',
 'Confusion',
 'Consciousness Disorders',
 'C

## Embeddings


In [20]:
from sentence_transformers import SentenceTransformer

In [21]:
embeddings_model = SentenceTransformer('all-mpnet-base-v2')

### Embeddings each symptom of each disease


In [22]:
symptoms = seperated_symptoms_each_disease  # your full list
n = len(symptoms)
third = n // 3

first_part = symptoms[:third]
second_part = symptoms[third:2*third]
third_part = symptoms[2*third:]

In [23]:
len(seperated_symptoms_each_disease), len(first_part), len(second_part), len(third_part)

(4219, 1406, 1406, 1407)

In [24]:
# # Start the multi-process pool on all available CUDA devices
# pool = embeddings_model.start_multi_process_pool()

# # Compute the embeddings using the multi-process pool
# all_symptoms_embeddings = embeddings_model.encode_multi_process(seperated_symptoms_each_disease, pool, show_progress_bar=True)
# print("Embeddings computed. Shape:", all_symptoms_embeddings.shape)

# embeddings_model.stop_multi_process_pool(pool)

# len(all_symptoms_embeddings), len(all_symptoms_embeddings[0])

In [25]:
# all_symptoms_embeddings = embeddings_model.encode(seperated_symptoms_each_disease, show_progress_bar=True)

# len(all_symptoms_embeddings), len(all_symptoms_embeddings[0])